# Overview

In this exercise we will post-process BAM files and perform SNP calling in those BAM files. The post-processing is necessary to reduce the high number of potential false SNPs that will get called. You will try to: 

- Align Illumina paired end reads 
- Sort and index BAM files
- Remove read duplicates from the BAM files
- Merge BAM files
- Use GATK to perform Base Quality Recalibration
- SNP calling using HaplotypeCaller (GATK4)
- Be acquainted with VCF-files

## Preparation

We are going to work on data from an Asian individual (HG00418) that was sequenced to around ~40X using Illumina Paired end sequencing. We are not going to use all the data, but only some data from two libraries that are both paired end.

One of the libraries are with an insert size of 180 and the other 481 and they are named "A" and "B" in the exercise below.

Files required for the exercises are listed below. Full paths are provided in the commands. You do not need to download the files.:
HG00418_A.bam and HG00418_B.bam
/exercises/alignment/blast/chr21.fasta
/exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.dbsnp.vcf.gz 
/exercises/alignment/blast/hg38bundle/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz 

Finally, we will use the tool GATK which can be found on the location below:
/exercises/software/gatk-4.1.7.0/gatk-package-4.1.7.0-local.jar
The commands for running GATK take several arguments and are long. We can run the jar files of the gatk commands as follows:
``java -Xmx2g -jar /exercises/software/gatk-4.1.7.0/gatk-package-4.1.7.0-local.jar -argument1 -argument2 -argument...


In [ ]:
! mkdir -p exercises/alignment/human
%cd exercises/alignment/human
! pwd

## Illumina Paired end reads

Finally, let us look at some paired end Illumina reads, these reads are from a ~40X wgs of an Asian individual. There are two different libraries - one called "A" and another library called "B". To save time we are not going to use all reads, we are only going to map them to chr21 and additionally the reads are already filtered so that we only have reads that map chr21. Create a directory for the exercise and make symbolic links to the data you need to there. Recall that we trimmed the "A" reads in the preprocessing exercise. 

If you did not manage to trim the A reads in the first exercise, then you can find the trimed files in the location below: /exercises/alignment/human/HG00418_A_1.trim.fastq-common.out.gz
/exercises/alignment/human/HG00418_A_2.trim.fastq-common.out.gz


Let's start the alignment, start by indexing chr21 (we have done this for you using the command ``bwa index chr21.fasta``) and then afterwards align the paired end files of library A. We are going to add something called a read group. The read group writes information on unique ID for the lane, which sample it is from, which platform was used etc.

To add a read group, you use the -R flag with BWA mem. For example, here you should use -R "@RG\tID:libA\tSM:HG00418\tPL:ILLUMINA" in the bwa mem command. The alignment will take a couple of minutes.


In [ ]:
! bwa mem -R "@RG\tID:libA\tSM:HG00418\tPL:ILLUMINA" /exercises/alignment/human/chr21.fasta /exercises/alignment/human/HG00418_A_1.trim.fastq-common.out.gz /exercises/alignment/human/HG00418_A_2.trim.fastq-common.out.gz > HG00418_A.sam

Remember that we create a BAM file from the SAM file. It is more efficient to work with the compressed BAM files and we save storage space.

In [ ]:
! samtools view -Sbh HG00418_A.sam > HG00418_A.bam

We align also the trimmed data of the file HG00418_B that we have trimmed and provide to you. The path for the file name is provided here:  /exercises/alignment/human/HG00418_B*.trim.fastq-common.out.gz. You do not need to do anything since it is already on the command below.

In [ ]:
! bwa mem -R "@RG\tID:libA\tSM:HG00418\tPL:ILLUMINA" /exercises/alignment/human/chr21.fasta /exercises/alignment/human/HG00418_B_1.trim.fastq-common.out.gz /exercises/alignment/human/HG00418_B_2.trim.fastq-common.out.gz > HG00418_B.sam

Once again, we create the BAM file from the SAM file for HG00418_B.sam file. Then we sort and index the file.

In [ ]:
! samtools view -Sbh HG00418_B.sam >HG00418_B.bam

## Mark Duplicates

Duplicates arise from artefacts during PCR amplification. What we will see in the alignment is that there will be many exact duplicates of a read. Because all the duplicated reads were sampled from the same DNA molecule it gives an uneven representation of that molecule compared to other molecules, and will bias the SNP calling. We will therefore remove them using Picard MarkDuplicates. Here duplicates are identified as reads that map with identical 5' coordinates and orientations, and only the reads with the highest quality score are kept for further analysis. We set REMOVE_DUPLICATES=true so that duplicate reads are removed from the BAM file, else they will just be flagged as duplicates by adding 1024 to their flag value (column 2). 

Begin by sorting the BAM file using samtools. This sorts the alignments by chromosome position instead of the order they appeared in the fastq files. Sorting is done with samtools sort <input.bam> <output_prefix>


In [ ]:
! samtools sort  -o HG00418_A.sort.bam HG00418_A.bam
! samtools index HG00418_A.sort.bam

In [ ]:
! samtools sort -o   HG00418_B.sort.bam HG00418_B.bam
! samtools index HG00418_B.sort.bam

<br>
Run the ``picard-tools MarkDuplicates`` command to remove duplicates in both files HG00418_A.sort.bam and HG00418_B.sort.bam (you can change the command accordingly if you changed the filename of the sorted bam file):


In [ ]:
! java -Xms2G -Xmx2G -jar /usr/share/java/picard.jar MarkDuplicates INPUT=HG00418_A.sort.bam OUTPUT=HG00418_A.sort.rmdup.bam ASSUME_SORTED=TRUE METRICS_FILE=/dev/null VALIDATION_STRINGENCY=SILENT REMOVE_DUPLICATES=true

In [ ]:
! java -Xms2G -Xmx2G -jar /usr/share/java/picard.jar MarkDuplicates INPUT=HG00418_B.sort.bam OUTPUT=HG00418_B.sort.rmdup.bam ASSUME_SORTED=TRUE METRICS_FILE=/dev/null VALIDATION_STRINGENCY=SILENT REMOVE_DUPLICATES=true

<br>
 
**Q1. How many reads ("records") are marked as duplicates from the files (hint: view the on-screen output from the two commands above)?**

It was not that many reads after all. However for e.g. ancient DNA or when the library is "over-sequenced" (i.e. low DNA input, many reads produced) this can easily be 20-50% or even 99% of all reads in a library. 

## Merge libraries to sample level

The rest of the analyses can run using just one BAM file per sample. Therefore we will merge the two libraries (here called HG00418_A.sort.rmdup.bam and HG00418_B.sort.rmdup.bam) and index the output BAM: 



In [ ]:
! samtools index HG00418_A.sort.rmdup.bam
! samtools index HG00418_B.sort.rmdup.bam

In [ ]:
! java -Xms2G -Xmx2G -jar /usr/share/java/picard.jar MergeSamFiles INPUT=HG00418_A.sort.rmdup.bam INPUT=HG00418_B.sort.rmdup.bam OUTPUT=HG00418.sort.rmdup.bam 


Now index the output file using samtools index. Note that this does not create a Burrows-Wheeler tranform... it doesn't make sense to BWT a BAM file!

In [ ]:
# Check what is your current directory. If it is not /home/jupyter-ngs-user*/exercises/alignment/human/
# then RESTART the kernel and run the first cell again. Then return to the exercise at this point.
! pwd

## Processing using GATK

Now we start using the Genome Analysis Toolkit (GATK) developed by the Broad Institute. It is one of the leading programs for processing BAM files and SNP calling. GATK is a java program and is called as by the main program .jar file and then each function is provided after the jar file.


### Base Quality Recalibration

The recalibration of the base qualities takes some time to run! Another important issue is that this only works when you have human data or another species where the variation is well described. 

Now we will recalibrate the base quality scores of the reads. This is because the base qualities assigned by the machine may not truely reflect the base calling errors and will lead to potential wrong variant calls. For this we will use known SNPs from dbSNP (database of known human variants) and Mills gold standard of indels as sites that are known to be variable in humans - those were the files we made links to in the beginning. 

We are going to recalibrate the reads based on quality scores, position in read (cycle) and dinucleotide content. First we need to count the statistics and output the sites that needs to be recalibrated. We will create files "recal_data.grp" with the information needed to recalibrate our quality scores. 

In [ ]:
%%bash
java -Xmx2g -DGATK_STACKTRACE_ON_USER_EXCEPTION=true -jar \
         /exercises/software/gatk-4.1.7.0/gatk-package-4.1.7.0-local.jar \
        BaseRecalibrator -I HG00418.sort.rmdup.bam \
        -R /exercises/alignment/human/chr21.fasta   \
        --known-sites /exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.dbsnp.vcf.gz \
        --known-sites /exercises/alignment/blast/hg38bundle/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz \
        -O recal_data.grp

Now we are going to recalibrate our reads in the alignment using the data that we created using the function ApplyBQSR. The function will output a new bam file with the recalibrated quality scores.

In [ ]:
%%bash 
java -Xmx2g -DGATK_STACKTRACE_ON_USER_EXCEPTION=true -jar \
         /exercises/software/gatk-4.1.7.0/gatk-package-4.1.7.0-local.jar \
        ApplyBQSR -R /exercises/alignment/human/chr21.fasta  \
       -I HG00418.sort.rmdup.bam \
       --bqsr-recal-file recal_data.grp \
       -O HG00418.sort.rmdup.recal.bam

# SNP calling 

In this part of the exercise we will use the processed bam-file and call SNPs from it. We will use GATK HaplotypeCaller for calling the SNPs. The output is a so-called VCF (variant call format) file. Information about the vcf file can be found __[here]( https://samtools.github.io/hts-specs/VCFv4.2.pdf)__.

## The HaplotypeCaller (GATK)

The current state-of-the-art genotyper is the HaplotypeCaller by the GATK. The HaplotypeCaller will perform a local de novo assembly around each of the potential variants in the alignment file and then output both SNPs and indels with very high accuracy. 
GATK is a Java program. We add an extra options: "--dbsnp" this will annotate our SNPs with known human variation ids, the output file "-O" which is output in VCF format, "-stand-call-conf" which is minimum Phred quality value of SNP to pass filtering. 
NB: The command takes around 10-15 minutes to run, so if you are in a hurry you can copy our file instead - see below if you want to do that. 

In [ ]:
%%bash
java -Xmx2g -DGATK_STACKTRACE_ON_USER_EXCEPTION=true -jar  \
        /exercises/software/gatk-4.1.7.0/gatk-package-4.1.7.0-local.jar HaplotypeCaller \
        -R  /exercises/alignment/human/chr21.fasta  \
        -I HG00418.sort.rmdup.recal.bam \
        --dbsnp /exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.dbsnp.vcf.gz \
        -O HG00418.raw.vcf.gz 

In [ ]:
# IF YOU ARE IN A HURRY COPY OUR VCF FILE to your working directory: 
# file can be found here: /exercises/alignment/human/A_reads/HG00418.raw.vcf.gz

Open the output with the raw SNP and indel calls (given by -O). You will see that the file consists of two main sections, header denoted by "#" as the first character and the actual SNP calls. The header gives you information about all the abbreviations in the file and is quite handy. The columns in the variant part correspond to: Chromosome, Position, dbSNP id, Reference base, Alternative base, Phred quality of call, Filter status, Lots of info, Format of the next field and then genotype information for our sample (HG00148).Let's try to see how many SNP/indel calls we got. Look at the file: zcat HG00418.raw.vcf.gz | head


In [ ]:
! zcat HG00418.raw.vcf.gz | head -50

Let's find out how many SNPs and indels have been identified. We will use the tool bcftools stats to get a summary of the vcf file. And we will use grep to extract the information we are interested in.<br> **Q2. a.How SNPs did we identify in our file?** <br>**b.How many indels?**

In [ ]:
! bcftools stats HG00418.raw.vcf.gz  | egrep "SN.*\snumber of SNPs" | cut -d"0" -f2
! bcftools stats HG00418.raw.vcf.gz  | egrep "SN.*\snumber of indels" | cut -d"0" -f2


Let us for now just use the SNPs and disregard the indels. We want to determine the total number of SNPs  that pass the criteria of quality (QUAL) greater than 20 and depth greater than 10.

In [ ]:
! bcftools filter -i 'TYPE="snp" && MIN(DP)>10 && QUAL>20' HG00418.raw.vcf.gz | bcftools stats | head -30

After running the above command answer the following questions: <br> **Q3. How many SNPs are meeting the criteria of having quality greater than 20 and depth greater than 10?**